In [2]:
%pip install faiss-cpu supervision -q

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
image_dir = 'images'
gallery_dir = 'images/gallery_paintings'

list_imgs = sorted(os.listdir(gallery_dir))

In [4]:
from torchvision import datasets, transforms
import torch

# create dataloader with required transforms 
tc = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()              
    ])

image_datasets = datasets.ImageFolder(image_dir, transform=tc)
dloader = torch.utils.data.DataLoader(image_datasets, batch_size=10, shuffle=False)

/Users/jaimielee/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jaimielee/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jaimielee/Library/Python/3.9/lib/python/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [5]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50')

# Select the desired layer
layer = model._modules.get('avgpool')

outputs = []
def copy_embeddings(m, i, o):
    """Copy embeddings from the penultimate layer.
    """
    o = o[:, :, 0, 0].detach().numpy().tolist()
    outputs.append(o)

# attach hook to the penulimate layer
_ = layer.register_forward_hook(copy_embeddings)

# Generate image's embeddings for all images in dloader and saves 
# them in the list outputs
for img, _ in dloader:
    _ = model(img)

list_embeddings = [item for sublist in outputs for item in sublist]

Using cache found in /Users/jaimielee/.cache/torch/hub/pytorch_vision_v0.10.0


In [6]:
len(list_embeddings[1])

2048

In [7]:
import base64
import os
from io import BytesIO
import cv2
import faiss
import numpy as np
import requests
from PIL import Image
import json
import supervision as sv

In [8]:
index = faiss.IndexFlatL2(2048)

list_embeddings = np.stack(list_embeddings)
index.add(list_embeddings)

# for embedding in list_embeddings:
#     index.add(np.array(embedding).astype(np.float32).reshape(2048,1))

faiss.write_index(index, "index.bin")

with open("index.json", "w") as f:
 	json.dump(list_imgs, f)

In [9]:
test_dir = 'test'
test_datasets = datasets.ImageFolder(test_dir, transform=tc)
dloader = torch.utils.data.DataLoader(test_datasets, batch_size=10, shuffle=False)

outputs = []
for img, _ in dloader:
    _ = model(img)
test_embedding = [item for sublist in outputs for item in sublist]
print(len(test_embedding))
print(len(test_embedding[0]))

1
2048


In [15]:
RESULTS_NUM = 2

query = test_embedding[0]
# test_embedding = np.stack(test_embedding)
D, I = index.search(np.array(query).reshape(1,2048), RESULTS_NUM)

images = [cv2.imread(os.path.join(gallery_dir, list_imgs[i])) for i in I[0]]

sv.plot_images_grid(images, (3, 3))

: 

: 